In [17]:
# import spacy
import re
# from spacy.tokens import DocBin
from fuzzywuzzy import fuzz
import psutil
import pandas as pd
import recordlinkage as rl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pyarrow
import dask.dataframe as dd
import pandas as pd
# get CPU count
import multiprocessing
# import Doc
# from spacy.tokens import Doc

def clean_cols(df):
    '''
    use snaky naming
    '''
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    return df

def get_doc_bin_custom(data, nlp):
    db = DocBin()
    for text, annotations in data:
        doc = Doc(nlp.vocab, words=text.split())
        # print("Text:", text)
        # print("Annotations:", annotations)
        ents = []
        for start, end, label in annotations['entities']:
            span = doc.char_span(start, end, label=label)
            if span is not None:
                print("Span:", span)
                ents.append(span)
            else:
                print("Warning: No span found for", start, end, label)
        doc.ents = ents
        db.add(doc)
    return db

stop_words = ['SPECIFICATION', 'PROVISIONAL', 'PROVISION', 'COMPLETE', '"']

def clean_text(df):
    '''
    clean text
    '''
    # use RE to replace between "IN THE" and "OF"
    df['description'] = df['description'].apply(lambda x: re.sub(r'IN THE \w+ OF', '', x))
    for stop_word in stop_words:
        df.description = df.description.str.replace(stop_word, '')
    df.description = df.description.str.replace('(', '').str.replace(')', '').str.replace(',', ' ').str.replace('  ', ' ').str.replace('  ', ' ').str.strip()
    
    # the same for address_1  address_2 address_3
    for col in ['address_1', 'address_2', 'address_3']:
        if col not in df.columns:
            continue
        # make it string
        df[col] = df[col].astype(str)
        # use re
        df[col] = df[col].apply(lambda x: re.sub(r'IN THE \w+ OF', '', x))
        df[col] = df[col].str.replace('(', '').str.replace(')', '').str.replace(',', ' ').str.replace('  ', ' ').str.replace('  ', ' ').str.strip()

    return df

def find_best_match_substring(search_text, query):
    words = search_text.split()
    best_match = None
    highest_score = 0
    best_start = None
    best_end = None

    # Iterate over all possible substrings
    for start in range(len(words)):
        for end in range(start, min(len(words), start + 10)):  # Limiting the length of substring
            substring = ' '.join(words[start:end+1])
            len_diff = abs(len(query) - len(substring))
            len_resemblance = 1 / (len_diff + 1)  # Normalized length difference

            score = fuzz.ratio(substring, query) * len_resemblance

            if score > highest_score:
                highest_score = score
                best_match = substring
                best_start = start
                best_end = end

            if highest_score > 95:  # Threshold for a good match
                break

    # Convert word indices to character indices
    if best_match is None:
        return None, None, None, None
    
    start_idx = len(' '.join(words[:best_start])) + (1 if best_start > 0 else 0)
    end_idx = len(' '.join(words[:best_end + 1]))

    return start_idx, end_idx, best_match, highest_score

# Function to report RAM usage
def report_ram_usage():
    process = psutil.Process()
    ram_usage = process.memory_info().rss / (1024 ** 2)  # in MB
    print(f"Current RAM usage: {ram_usage:.2f} MB")

# Report RAM before starting
report_ram_usage()

Current RAM usage: 11693.85 MB


## Stopwords finder

In [2]:
# patent = pd.read_parquet('../../Patents/data/patent_clean.parq')

# # find words of highest frequencies in addr_to_use
# from collections import Counter

# desc_concated = " ".join(e for e in patent.description.tolist() if e is not None)
# Counter(desc_concated.split()).most_common(100)

## Read in

In [3]:
# # import local culaster from dsak
# from dask.distributed import LocalCluster

# client = LocalCluster(n_workers = 8, threads_per_worker = 4)

In [4]:
# parsed addr from patents
# patent_aut = pd.read_csv("../output/aut_adr_cleaned_partial.csv")
# final_prepared = dd.read_parquet('../output/all_matched_census_name_address_age.parquet')
# concat pname oname sname to name, if not NULL or None
# census_sampled = final_prepared.sample(frac=0.0001, random_state=42).compute(client=client)
# census_sampled

patent_aut = pd.read_csv("../output/aut_adr_cleaned_partial_toy.csv")
final_prepared = pd.read_csv('../output/all_matched_census_name_address_age_toy.csv')
census_sampled = final_prepared

In [5]:
final_prepared.columns

Index(['Unnamed: 0', 'recid', 'pname', 'oname', 'sname', 'address', 'age',
       'parishuk_1881', 'division', 'county', 'district', 'subdist',
       'ConParID_18511911', 'year', 'parishuk_1911', 'parishuk_1861',
       'parishuk_1901', 'parishuk_1891', 'parishuk_1851'],
      dtype='object')

In [6]:
!ls -lha ../output/all_matched_census_name_address_age_toy.csv

-rw-r--r-- 1 xiet13 cluster-users 96K Mar  1 15:55 ../output/all_matched_census_name_address_age_toy.csv


In [7]:
patent_aut

,i,name_0,addr_0,name_1,addr_1,name_2,addr_2
0,GB188002025,WATSON SMITH,LEY THECAPABLE OF REMOVING THE FREE HYDROCHLOR...,NaN,NaN,NaN,NaN
1,GB187600824,GEORGE NEWTON,11 AN IMPROVED LUBRICANT THIS INVENTION HAS FO...,NaN,NaN,NaN,NaN
2,GB187303416,JAMES HARVEY,OF HILLSIDE ESTATE VERE JAMAICA AND OF,JAMES HARVEY,OF HILLSIDE ESTATE VERE JAMAICA AND OF,NaN,NaN
3,GB188907812,GEORGE ROSE,WELLINGTON STREET ARCHIBALD BAIRD STEEL MANUFA...,NaN,NaN,NaN,NaN
4,GB187102200,ROBERT CARTER MOFFAT,AND ALEXANDER MCLAREN AT THE OFFICE OF THE COM...,LANARK NORTH BRITAIN,AND ALEXANDER MCLAREN BOTH,ROBERT CARTER MOFFAT,AND ALEXANDER MCLAREN BOTH
...,...,...,...,...,...,...,...
193,GB189408988,TEAJAMES WILLIAM HAME,13 ST GILES STREET NORWICH NORFOK,NaN,NaN,NaN,NaN
194,GB190927499,NaN,NaN,NaN,NaN,NaN,NaN
195,GB190108920,JAMES ZIITNST,GOLDHAWK ROAD,NaN,NaN,NaN,NaN
196,GB190309281,JOSEPH BEES OF,GLAMORGAN COLLIER,NaN,NaN,NaN,NaN


In [8]:
patent_aut.shape, final_prepared.shape

((198, 7), (882, 19))

#### Interlude: examine uniqueness of 'name', 'address'

In [9]:
census_sampled.oname = census_sampled.oname.fillna('').apply(str.strip)
census_sampled['pname'] = census_sampled['pname'].astype('str')
census_sampled['oname'] = census_sampled['oname'].astype('str')
census_sampled['sname'] = census_sampled['sname'].astype('str')
census_sampled['name'] = census_sampled.apply(lambda row:row.pname + ' ' + row.oname + ' ' + row.sname if row.oname != '' else row.pname + ' ' + row.sname, axis = 1).astype('str')
census_sampled['address'] = census_sampled['address'].astype('str')
census_sampled['name_address'] = (census_sampled['name'].apply(str.strip) + ' ' + census_sampled.address.apply(str.strip)).apply(str.strip)

In [10]:
# census_sampled[['name', 'address']].nunique(), pd.DataFrame(census_sampled['name'] + census_sampled['address'])[0].nunique()
# # (name       7608
# #  address    7403
# #  dtype: int64,
# #  8044)

mask_name_repeated = census_sampled.groupby('name')['name'].transform('count') > 1
mask_address_repeated = census_sampled.groupby('address')['address'].transform('count') > 1
# # Apply the mask to filter rows with repeated names
census_sampled[mask_name_repeated | mask_address_repeated].shape[0], census_sampled[mask_name_repeated & mask_address_repeated].shape[0] # 65 
# # (1511, 65)


(316, 154)

In [11]:
# census_sampled['name_address']
mask_name_address_repeated = census_sampled.groupby('name_address')['name_address'].transform('count') > 1
census_sampled[mask_name_address_repeated][['name',	'name_address', 'county',	'district',	'subdist',	'ConParID_18511911',	'year']]

,name,name_address,county,district,subdist,ConParID_18511911,year
0,nan nan,nan nan @@@,LONDON,HOLBORN,GOSWELL STREET,100002.0,1881
1,nan nan,nan nan @@@,LONDON,HOLBORN,GOSWELL STREET,100002.0,1881
14,GEORGE GRANT,GEORGE GRANT COTTAGE,HAMPSHIRE,WINCHESTER,TWYFORD,100774.0,1881
15,GEORGE GRANT,GEORGE GRANT COTTAGE,HAMPSHIRE,WINCHESTER,TWYFORD,100774.0,1881
24,CHARLES STANTON,CHARLES STANTON @@@,BEDFORDSHIRE,BIGGLESWADE,POTTON,102067.0,1881
...,...,...,...,...,...,...,...
832,ROBERT COLLINS,ROBERT COLLINS KINGSON,DORSETSHIRE,STURMINSTER,STURMINSTER,103927.0,1851
851,KEZIA ASHTON,KEZIA ASHTON VILLAGE,LINCOLNSHIRE,SPILSBY,SPILSBY,106933.0,1851
852,KEZIA ASHTON,KEZIA ASHTON VILLAGE,LINCOLNSHIRE,SPILSBY,SPILSBY,106933.0,1851
866,WILLIAM COLBECK,WILLIAM COLBECK NEWMILLER DAM,YORKSHIRE WEST RIDING,WAKEFIELD,SANDAL,108627.0,1851


In [12]:
repeat_rate = census_sampled[mask_name_address_repeated].shape[0] / census_sampled.shape[0]
repeat_rate

0.16666666666666666

#### Interlude finished

In [13]:
# replace None with '' in oname and turn to string
final_prepared['oname'] = final_prepared['oname'].apply(lambda x: '' if x is None else x).astype(str)
final_prepared['name'] = final_prepared['pname'] + ' ' + final_prepared['oname'] + ' ' + final_prepared['sname']
final_prepared = final_prepared.drop(columns=['pname', 'oname', 'sname'])

In [14]:
patent_aut.set_index('i', inplace=True)

# split into 3 dfs with name and addr
patent_aut1 = patent_aut[['name_0', 'addr_0']].rename(columns={'name_0': 'name', 'addr_0': 'addr'})
patent_aut2 = patent_aut[['name_1', 'addr_1']].rename(columns={'name_1': 'name', 'addr_1': 'addr'})
patent_aut3 = patent_aut[['name_2', 'addr_2']].rename(columns={'name_2': 'name', 'addr_2': 'addr'})

# concat vertically
patent_aut = pd.concat([patent_aut1, patent_aut2, patent_aut3], axis=0).reset_index()
# clean wherer name is None or NaN
patent_aut = patent_aut[patent_aut.name.notnull()]
patent_aut

,i,name,addr
0,GB188002025,WATSON SMITH,LEY THECAPABLE OF REMOVING THE FREE HYDROCHLOR...
1,GB187600824,GEORGE NEWTON,11 AN IMPROVED LUBRICANT THIS INVENTION HAS FO...
2,GB187303416,JAMES HARVEY,OF HILLSIDE ESTATE VERE JAMAICA AND OF
3,GB188907812,GEORGE ROSE,WELLINGTON STREET ARCHIBALD BAIRD STEEL MANUFA...
4,GB187102200,ROBERT CARTER MOFFAT,AND ALEXANDER MCLAREN AT THE OFFICE OF THE COM...
...,...,...,...
400,GB187102200,ROBERT CARTER MOFFAT,AND ALEXANDER MCLAREN BOTH
445,GB189311397,BAKER AND FRANK MOORE,OF 2 FLORENCE VILLAS ALGAR
491,GB189311611,ARNULPH MALLOOK OF NO,1 FIT SOMEWHAT LOOSELY
504,GB190304313,LESLIE ROAD LEJ,3II < ELL LONDON


## Soundex

In [15]:
from recordlinkage.preprocessing import clean, phonetic
# simple clean
patent_aut['addr'] = patent_aut.addr.apply(str.upper)
# patent_aut['addr'] = patent_aut.addr.apply(eval)
patent_aut['name'] = patent_aut['name'].apply(str.upper)
patent_aut

,i,name,addr
0,GB188002025,WATSON SMITH,LEY THECAPABLE OF REMOVING THE FREE HYDROCHLOR...
1,GB187600824,GEORGE NEWTON,11 AN IMPROVED LUBRICANT THIS INVENTION HAS FO...
2,GB187303416,JAMES HARVEY,OF HILLSIDE ESTATE VERE JAMAICA AND OF
3,GB188907812,GEORGE ROSE,WELLINGTON STREET ARCHIBALD BAIRD STEEL MANUFA...
4,GB187102200,ROBERT CARTER MOFFAT,AND ALEXANDER MCLAREN AT THE OFFICE OF THE COM...
...,...,...,...
400,GB187102200,ROBERT CARTER MOFFAT,AND ALEXANDER MCLAREN BOTH
445,GB189311397,BAKER AND FRANK MOORE,OF 2 FLORENCE VILLAS ALGAR
491,GB189311611,ARNULPH MALLOOK OF NO,1 FIT SOMEWHAT LOOSELY
504,GB190304313,LESLIE ROAD LEJ,3II < ELL LONDON


In [16]:
mask_addr_len = patent_aut.addr.apply(lambda x: len(x) > 1)
mask_addr_len.value_counts()

addr
True    228
Name: count, dtype: int64

In [17]:
# merge what is inside addr: 
patent_aut['addr_to_use'] = patent_aut.addr
patent_aut.drop(columns = ['addr'], inplace = True)

In [18]:
import recordlinkage as rl 

# phonetic_methods = ['soundex', 'nysiis', 'metaphone', 'match_rating']
phonetic_methods = ['soundex', 'metaphone']

for method in phonetic_methods:
    print(method)
    patent_aut[f'name_{method}'] = phonetic(patent_aut['name'], method = method)

patent_aut

soundex
metaphone


,i,name,addr_to_use,name_soundex,name_metaphone
0,GB188002025,WATSON SMITH,LEY THECAPABLE OF REMOVING THE FREE HYDROCHLOR...,W325,WTSNSM0
1,GB187600824,GEORGE NEWTON,11 AN IMPROVED LUBRICANT THIS INVENTION HAS FO...,G625,JRJNTN
2,GB187303416,JAMES HARVEY,OF HILLSIDE ESTATE VERE JAMAICA AND OF,J526,JMXRF
3,GB188907812,GEORGE ROSE,WELLINGTON STREET ARCHIBALD BAIRD STEEL MANUFA...,G626,JRJRS
4,GB187102200,ROBERT CARTER MOFFAT,AND ALEXANDER MCLAREN AT THE OFFICE OF THE COM...,R163,RBRTKRTRMFT
...,...,...,...,...,...
400,GB187102200,ROBERT CARTER MOFFAT,AND ALEXANDER MCLAREN BOTH,R163,RBRTKRTRMFT
445,GB189311397,BAKER AND FRANK MOORE,OF 2 FLORENCE VILLAS ALGAR,B265,BKRNTFRNKMR
491,GB189311611,ARNULPH MALLOOK OF NO,1 FIT SOMEWHAT LOOSELY,A654,ARNLFMLKFN
504,GB190304313,LESLIE ROAD LEJ,3II < ELL LONDON,L246,LSLRTLJ


In [19]:
final_prepared.name = final_prepared.name.apply(str.upper)
for method in phonetic_methods:
    print(method)
    final_prepared[f'name_{method}'] = phonetic(final_prepared['name'], method = method)

soundex
metaphone


In [20]:
for method in phonetic_methods:
    print(patent_aut[f'name_{method}'].nunique())

144
225


## Address hack 
(syntactic pure addres + semantic concated addr (addr + parish + ...))

In [21]:
final_prepared['addr_to_use'] = final_prepared['address']

## Create Candidates

In [23]:
final_prepared = final_prepared.reset_index().drop_duplicates(subset= ['recid', 'year']).set_index(['recid', 'year'])
# patent_aut.set_index('i', inplace=True)

In [24]:
patent_aut

,i,name,addr_to_use,name_soundex,name_metaphone
0,GB188002025,WATSON SMITH,LEY THECAPABLE OF REMOVING THE FREE HYDROCHLOR...,W325,WTSNSM0
1,GB187600824,GEORGE NEWTON,11 AN IMPROVED LUBRICANT THIS INVENTION HAS FO...,G625,JRJNTN
2,GB187303416,JAMES HARVEY,OF HILLSIDE ESTATE VERE JAMAICA AND OF,J526,JMXRF
3,GB188907812,GEORGE ROSE,WELLINGTON STREET ARCHIBALD BAIRD STEEL MANUFA...,G626,JRJRS
4,GB187102200,ROBERT CARTER MOFFAT,AND ALEXANDER MCLAREN AT THE OFFICE OF THE COM...,R163,RBRTKRTRMFT
...,...,...,...,...,...
400,GB187102200,ROBERT CARTER MOFFAT,AND ALEXANDER MCLAREN BOTH,R163,RBRTKRTRMFT
445,GB189311397,BAKER AND FRANK MOORE,OF 2 FLORENCE VILLAS ALGAR,B265,BKRNTFRNKMR
491,GB189311611,ARNULPH MALLOOK OF NO,1 FIT SOMEWHAT LOOSELY,A654,ARNLFMLKFN
504,GB190304313,LESLIE ROAD LEJ,3II < ELL LONDON,L246,LSLRTLJ


In [25]:

# for repeated index, add another level of index starting from 0
patent_aut.groupby('i').cumcount()
# patent_aut
# make into multiindex
patent_aut.set_index(['i', patent_aut.groupby('i').cumcount()], inplace=True)

# rename the second level of index as count
patent_aut.index.rename(['i', 'count'], inplace=True)

In [26]:
assert patent_aut.index.is_unique
# patent_aut.index.rename(['i', 'count'], inplace=True)
# patent_aut

# save to parquet
# patent_aut.to_parquet('../Output/patent_for_match_90315_ckpt_rl.parquet')
# assert final_prepared.index.is_unique
# final_prepared.to_parquet('../Output/final_prepared_ckpt_rl.parquet')

### ReadIn_new

In [27]:
# del idx
import pandas as pd
# final_prepared = pd.read_parquet('../output/final_prepared_ckpt_rl.parquet') # Used to crushed kernel
# final_prepared = final_prepared.drop_duplicates(subset=['recid', 'year'])
assert final_prepared.index.is_unique
# patent_aut = pd.read_parquet('../output/patent_for_match_90315_ckpt_rl.parquet')

In [28]:
final_prepared

,,index,Unnamed: 0,address,age,parishuk_1881,division,county,district,subdist,ConParID_18511911,parishuk_1911,parishuk_1861,parishuk_1901,parishuk_1891,parishuk_1851,name,name_address,name_soundex,name_metaphone,addr_to_use
recid,year,,,,,,,,,,,,,,,,,,,,
972527,1881,0,0,@@@,28.0,30.0,I,LONDON,HOLBORN,GOSWELL STREET,100002.0,NaN,NaN,NaN,NaN,NaN,NAN NAN,nan nan @@@,N550,NNN,@@@
594589,1881,2,2,151 GROSVENOR RD,33.0,14.0,I,LONDON,ST GEORGE HANOVER SQUARE,BELGRAVE,100001.0,NaN,NaN,NaN,NaN,NaN,MARIA COE,MARIA COE 151 GROSVENOR RD,M620,MRK,151 GROSVENOR RD
1705473,1881,3,3,72 KINGS CROSS RD,26.0,61.0,I,LONDON,HOLBORN,GOSWELL STREET,100002.0,NaN,NaN,NaN,NaN,NaN,W HUTCHINSON,W HUTCHINSON 72 KINGS CROSS RD,W325,WXNSN,72 KINGS CROSS RD
1147891,1881,4,4,234 CALEDONIAN RD,48.0,33.0,I,LONDON,HOLBORN,GOSWELL STREET,100002.0,NaN,NaN,NaN,NaN,NaN,LUISA E PERRINS,LUISA E PERRINS 234 CALEDONIAN RD,L216,LSPRNS,234 CALEDONIAN RD
1110913,1881,5,5,99 LIVERPOOL BUILDINGS,36.0,33.0,I,LONDON,HOLBORN,GOSWELL STREET,100002.0,NaN,NaN,NaN,NaN,NaN,ROBERT CAMMERON,ROBERT CAMMERON 99 LIVERPOOL BUILDINGS,R163,RBRTKMRN,99 LIVERPOOL BUILDINGS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16157026,1851,877,877,-,45.0,NaN,X,NORTHUMBERLAND,ROTHBURY,ELSDON,110223.0,NaN,NaN,NaN,NaN,14530.0,WILLIAM DAVISON,WILLIAM DAVISON -,W453,WLMTFSN,-
16251755,1851,878,878,-,62.0,NaN,X,CUMBERLAND,WIGTON,WIGTON,110312.0,NaN,NaN,NaN,NaN,14721.0,JOSEPH TWENTYMAN,JOSEPH TWENTYMAN -,J213,JSFTWNTMN,-
16300234,1851,879,879,TALLENTIRE,21.0,NaN,X,CUMBERLAND,COCKERMOUTH,MARYPORT,110367.0,NaN,NaN,NaN,NaN,14803.0,JOSEPH COCKTON,JOSEPH COCKTON TALLENTIRE,J212,JSFKKTN,TALLENTIRE


In [29]:
# import dask.dataframe as dd
# # final_prepared = dd.read_parquet('../Output/final_prepared_ckpt_rl.parquet')
# # dedupe
# final_prepared = final_prepared.reset_index().drop_duplicates(subset=['recid', 'year'], keep='first')
# final_prepared = final_prepared.set_index(['recid', 'year'])#  Dask dataframe does not yet support multi-indexes.

In [30]:
# final_prepared.to_parquet('../Output/final_prepared_ckpt_rl.parquet')

In [31]:
import recordlinkage as rl
# import  SortedNeighbourhoodIndex
from recordlinkage.index import SortedNeighbourhood
# initalise an index
indexer = rl.Index()


# blocking_columns = ['name_soundex'] # Preparing for Geo-blocking 
SortedNeighbourhood_columns = ['name_metaphone']
# patent_aut.name_soundex.nunique() # 4045


# indexer.block(left_on=blocking_columns, right_on=blocking_columns)
indexer.add(SortedNeighbourhood(left_on=SortedNeighbourhood_columns, right_on=SortedNeighbourhood_columns, window=3))
candidates = indexer.index(patent_aut, final_prepared)

In [32]:
report_ram_usage()

Current RAM usage: 355.43 MB


In [33]:
# # candidates = indexer.index(patent_aut, final_prepared)
# # candidates
# # process final_prepared in batch
# import os, pickle

# batch_size = 10000000
# save_dir = '../Output/temp/name_metaphone_addr_to_use'
# os.makedirs(save_dir, exist_ok=True)

# for i in range(0, len(final_prepared), batch_size):
#     save_file = f"pat_census_candidates_{i}.pkl"
#     save_path = os.path.join(save_dir, save_file)
#     if os.path.exists(save_path):
#         print(f"File {save_file} exists, continue")
#         continue
#     print(i)
#     final_prepared_batch = final_prepared.iloc[i: i + batch_size]
#     # candidates = indexer.index(patent_aut, final_prepared_batch.set_index(['recid', 'year'])) # for dd
#     candidates = indexer.index(patent_aut, final_prepared_batch)
#     report_ram_usage()
#     print("done")
#     # save candidates
#     with open(save_path, 'wb') as f:
#         pickle.dump(candidates, f)

#     # del candidates
#     # del final_prepared_batch

# # Report RAM after indexer initialization
# report_ram_usage()

In [34]:
# !ls -lha ../Output/temp/name_metaphone_addr_to_use/

In [35]:
# # read pattern: ../Output/temp/pat_census_candidates_**
# import os, pickle
# # save_dir = '../Output/temp/'
# save_dir = '../Output/temp/name_metaphone_addr_to_use'

# candidate_path = []
# for file in os.listdir(save_dir):
#     if file.startswith('pat_census_candidates'):
#         candidate_path.append(os.path.join(save_dir, file))

# # read candidates
# candidates = []
# for path in candidate_path:
#     with open(path, 'rb') as f:
#         candidates.append(pickle.load(f))


In [36]:
def get_four_level_idx(partition_result):
    # reset index with names patent_idx and census_idx
    partition_result = partition_result.reset_index().rename(columns={'level_0': 'patent_idx', 'level_1': 'census_idx'})
    # extract patent_id and patent_count from patent_idx
    partition_result['patent_id'] = partition_result['patent_idx'].apply(lambda x:x[0])
    partition_result['patent_count'] = partition_result['patent_idx'].apply(lambda x:x[1])
    # extract recid and year from census_idx
    partition_result['recid'] = partition_result['census_idx'].apply(lambda x:x[0])
    partition_result['year'] = partition_result['census_idx'].apply(lambda x:x[1])
    # drop patent_idx and census_idx
    partition_result.drop(columns=['patent_idx', 'census_idx'], inplace=True)
    partition_result = partition_result.set_index(['patent_id', 'patent_count', 'recid', 'year'])
    return partition_result

In [37]:
print("Initializing comparer...")
# comparer = rl.Compare(n_jobs = -1)  # Utilizing all available CPUs
comparer = rl.Compare()

str_methods = {
    'name': 'jarowinkler',
    # 'name_metaphone': 'jarowinkler',
    'addr_to_use': 'levenshtein'
}
features = str_methods.keys()
for attr, method in str_methods.items():
    comparer.string(attr, attr, method=method, label=f'{attr}_{method}')
partition_result = comparer.compute(candidates, patent_aut, final_prepared)
partition_result = get_four_level_idx(partition_result)
# read one from candidate_path, compute, and save

# comparing_batch_size = 10 ** 7
# version_name = '2features_mean_threshold_0.7'
# # mean_threshold = 0.7 # keep 0.6%

# os.makedirs(os.path.join(save_dir, version_name), exist_ok=True)

# for path in candidate_path:
#     starting_idx = int(path.split('_')[-1].split('.')[0])
#     print(f"Starting from {starting_idx}")

#     with open(path, 'rb') as f:
#         candidates = pickle.load(f)
    
#     # measure time
#     import time
#     start = time.time()
#     for i in range(0, len(candidates), comparing_batch_size):
#         save_file = f"{save_dir}/{version_name}/pat_census_mtpool_{starting_idx}_{i}.parquet"
#         print(f"save file: {save_file}")
#         if os.path.exists(save_file):
#             print(f"File {save_file} exists, continue")
#             continue
#         print(f"Processing {i} to {i + comparing_batch_size}")

#         partition_result = comparer.compute(candidates[i: i + comparing_batch_size], patent_aut, final_prepared)
#         # # initial filterting using mean
#         # print("Filtering...")
#         # # partition_result['mean'] = partition_result.mean(axis=1)
#         # mask_mean = partition_result.mean(axis=1) > mean_threshold
#         # partition_result = partition_result[mask_mean]
#         # print(f"Filtered {len(partition_result)}")
#         # save to parquet
#         get_four_level_idx(partition_result).to_parquet(save_file)
#         end = time.time()
#         print(f"Done {i} to {i + comparing_batch_size}, used {end - start} seconds")
#     print("done")

# # Add string comparison methods to comparer
# for attr, method in str_methods.items():
#     comparer.string(attr, attr, method=method, label=f'{attr}_{method}')

# partition_result = comparer.compute(candidates, patent_aut, final_prepared)

Initializing comparer...


In [38]:
# # read all ../Output/temp/3features/pat_census_mtpool_0_**
# import glob
# partition_result_list = []
# for path in glob.glob('../Output/temp/name_metaphone_addr_to_use/2features_mean_threshold_0.7/'):
#     partition_result_list.append(pd.read_parquet(path))
# partition_result = pd.concat(partition_result_list)

## Initial Look

In [39]:
partition_result

name_jarowinkler  \
patent_id   patent_count recid    year                     
GB188002025 0            1705473  1881          0.564408   
                         2303080  1911          0.564408   
                         4427159  1861          0.564408   
GB187600824 0            11166494 1891          0.856410   
GB188907812 0            8730659  1881          0.869519   
...                                                  ...   
GB190725421 1            14986608 1851          0.858462   
GB190604178 1            14471429 1881          0.689947   
GB190002764 1            16088027 1851          0.861183   
GB190304313 2            10550285 1891          0.516667   
GB190802899 2            12716557 1891          0.644661   

                                        addr_to_use_levenshtein  
patent_id   patent_count recid    year                           
GB188002025 0            1705473  1881                 0.140845  
                         2303080  1911                 0.225352  
                         4427159  1861                 0.169014  
GB187600824 0            11166494 1891                 0.118812  
GB188907812 0            8730659  1881                 0.115385  
...                                                         ...  
GB190725421 1            14986608 1851                 0.058824  
GB190604178 1            14471429 1881                 0.000000  
GB190002764 1            16088027 1851                 0.066667  
GB190304313 2            10550285 1891                 0.250000  
GB190802899 2            12716557 1891                 0.222222  

[430 rows x 2 columns]

In [40]:
partition_result['mean'] = partition_result.mean(axis=1)
partition_result['simple_weighted_score'] = 0.7*partition_result.name_jarowinkler + 0.3*partition_result.addr_to_use_levenshtein
# select with mean being quanitle 0.9
potential_answer = partition_result[partition_result['mean'] > partition_result['mean'].quantile(0.99)]

In [41]:
potential_answer.sort_values(by='mean', ascending=False)

,,,,name_jarowinkler,addr_to_use_levenshtein,mean,simple_weighted_score
patent_id,patent_count,recid,year,,,,
GB189311697,0,13179398,1911,0.985714,0.224490,0.605102,0.757347
GB189313221,0,5000132,1891,0.978947,0.218750,0.598849,0.750888
GB189311246,0,18725234,1891,0.947222,0.240000,0.593611,0.735056
GB189900345,0,5712820,1891,0.880000,0.291667,0.585833,0.703500
GB189313221,0,6545942,1891,0.871930,0.296296,0.584113,0.699240


In [42]:
potential_answer = potential_answer.reset_index().rename(columns = {'patent_count': 'count', 'patent_id':'i'})
potential_answer

,i,count,recid,year,name_jarowinkler,addr_to_use_levenshtein,mean,simple_weighted_score
0,GB189900345,0,5712820,1891,0.880000,0.291667,0.585833,0.703500
1,GB189311246,0,18725234,1891,0.947222,0.240000,0.593611,0.735056
2,GB189313221,0,5000132,1891,0.978947,0.218750,0.598849,0.750888
3,GB189311697,0,13179398,1911,0.985714,0.224490,0.605102,0.757347
4,GB189313221,0,6545942,1891,0.871930,0.296296,0.584113,0.699240


In [43]:
potential_answer = potential_answer.reset_index().rename(columns = {'patent_count': 'count', 'patent_id':'i'})
# potential_answer = potential_answer.rename(columns = {'i': 'count', 'patent_id':'i'})
# merge with final_prepared and patent_aut
potential_answer = potential_answer.merge(patent_aut.reset_index(), on = ['i', 'count'])
# final_prepared
potential_answer = potential_answer.merge(final_prepared.reset_index(), on = ['recid', 'year'])
# patent_aut


In [44]:
look1 = potential_answer[['count', 'i', 'recid', 'year', 'name_x', 'name_y', 'addr_to_use_x', 'addr_to_use_y', 'mean']].sort_values(by='mean', ascending=False).drop_duplicates(subset= ['count', 'i', 'year'], keep='first')
look1 = look1[['count', 'i', 'recid', 'year', 'name_x', 'name_y', 'addr_to_use_x', 'addr_to_use_y', 'mean']].sort_values(by='mean', ascending=False).drop_duplicates(subset= ['recid', 'year'], keep='first')
look1

,count,i,recid,year,name_x,name_y,addr_to_use_x,addr_to_use_y,mean
4,0,GB189311697,13179398,1911,HENRY HERBERT,HENRY HERBERT,154 ELTHORNE ROAD HORNSEY RISE DLIDDLESEX CABINET,BROAD BUSH BLUNSDON ST LEONARD,0.605102
2,0,GB189313221,5000132,1891,EDWARD WILLIAM LYNE,EDWARD WILLIAM LANE,203 ASHMORE ROAD PADDINGTON,BLACKLANDS FARM BLACKLANDS ROAD,0.598849
1,0,GB189311246,18725234,1891,ANNIE STEW ART,ANN STEWART,41 LEYLAND ROAD SOUTHPORT,ROSCOMMON STREET,0.593611
0,0,GB189900345,5712820,1891,HENRY VALENTINE,HENRY FLEET,2 VICTORIA STREET LONDON,THE GREEN,0.585833


In [45]:
look1[look1['mean'] > 0.8].to_csv('../output/pat_census_match_look1.csv')

In [46]:
look1[look1['mean'] > 0.6]

,count,i,recid,year,name_x,name_y,addr_to_use_x,addr_to_use_y,mean
4,0,GB189311697,13179398,1911,HENRY HERBERT,HENRY HERBERT,154 ELTHORNE ROAD HORNSEY RISE DLIDDLESEX CABINET,BROAD BUSH BLUNSDON ST LEONARD,0.605102


In [47]:
# partition_result['mean'].quantile(0.99) # 0.6923202614379086

In [48]:
# patent_part = patent_aut.loc[potential_answer.get_level_values(0)]
# census_part = census_sampled.loc[potential_answer.get_level_values(1)]

# # merge using potential_answer


In [49]:
# patent_part

In [50]:
# census_part

## Initial Selection

In [51]:
partition_result

name_jarowinkler  \
patent_id   patent_count recid    year                     
GB188002025 0            1705473  1881          0.564408   
                         2303080  1911          0.564408   
                         4427159  1861          0.564408   
GB187600824 0            11166494 1891          0.856410   
GB188907812 0            8730659  1881          0.869519   
...                                                  ...   
GB190725421 1            14986608 1851          0.858462   
GB190604178 1            14471429 1881          0.689947   
GB190002764 1            16088027 1851          0.861183   
GB190304313 2            10550285 1891          0.516667   
GB190802899 2            12716557 1891          0.644661   

                                        addr_to_use_levenshtein      mean  \
patent_id   patent_count recid    year                                      
GB188002025 0            1705473  1881                 0.140845  0.352626   
                         2303080  1911                 0.225352  0.394880   
                         4427159  1861                 0.169014  0.366711   
GB187600824 0            11166494 1891                 0.118812  0.487611   
GB188907812 0            8730659  1881                 0.115385  0.492452   
...                                                         ...       ...   
GB190725421 1            14986608 1851                 0.058824  0.458643   
GB190604178 1            14471429 1881                 0.000000  0.344974   
GB190002764 1            16088027 1851                 0.066667  0.463925   
GB190304313 2            10550285 1891                 0.250000  0.383333   
GB190802899 2            12716557 1891                 0.222222  0.433442   

                                        simple_weighted_score  
patent_id   patent_count recid    year                         
GB188002025 0            1705473  1881               0.437339  
                         2303080  1911               0.462691  
                         4427159  1861               0.445790  
GB187600824 0            11166494 1891               0.635131  
GB188907812 0            8730659  1881               0.643278  
...                                                       ...  
GB190725421 1            14986608 1851               0.618570  
GB190604178 1            14471429 1881               0.482963  
GB190002764 1            16088027 1851               0.622828  
GB190304313 2            10550285 1891               0.436667  
GB190802899 2            12716557 1891               0.517929  

[430 rows x 4 columns]

In [11]:
# selet top 10%
# partition_result['mean'] = partition_result.mean(axis = 1)
# partition_result['simple_weighted_score'] = 0.7*partition_result.name_jarowinkler + 0.3*partition_result.addr_to_use_levenshtein
# result = partition_result[partition_result['mean'] > partition_result['mean'].quantile(0.9)]

result = partition_result
# extra_features = pd.read_parquet("../output/MATCHES_recid_title_labforce_occstrng_age.parquet")
extra_features = pd.read_csv("../output/MATCHES_recid_title_labforce_occstrng_age_toy.csv")
extra_features

/tmp/ipykernel_28071/3206823909.py:8: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  extra_features = pd.read_csv("../output/MATCHES_recid_title_labforce_occstrng_age_toy.csv")


In [55]:
# result.reset_index().patent_id.nunique() # 66334

#### get extra_features

In [56]:
# # _1851_partial = dd.read_parquet("../Census_samples/Whole_ipum/Whole_1851", columns = columns_for_merge)

# # merge to use age restriction
# # potentially use labour force as well
# # columns_for_merge = ['recid', 'title', 'labforce', 'occstrng', 'age']
# columns_for_merge = ['recid', 'labforce', 'occstrng', 'age']

# dt = {i:None for i in range(1851, 1902, 10) if i != 1871}
# for i in dt.keys():
#     # if i == 1871: continue
#     columns_alt = ['recid', 'title', 'labforce', 'occstrng', f'uk{i}a_age']
#     try: 
#         dt[i] = dd.read_parquet(f"../Census_samples/Whole_ipum/Whole_{i}", columns = columns_alt)
#     except: 
#         dt[i] = dd.read_parquet(f"../Census_samples/Whole_ipum/Whole_{i}", columns = columns_for_merge)
#     finally:
#         dt[i] = dt[i].compute(worker = client)
# dt[1911] = pd.read_stata("../Census_samples/Whole_ipum/Census_IPUMS.dta", convert_categoricals=False)[['uk1911a_recid', 'labforce', 'occstrng', 'age']].rename(columns = {'uk1911a_recid': 'recid'})

In [57]:
# # prepare for merge
# for i in dt.keys():
#     dt[i]['year'] = str(i)
#     if f"uk{i}a_age" in dt[i].columns.tolist():
#         dt[i].rename(columns = {f"uk{i}a_age":"age"}, inplace = True)
# extra_features = pd.concat(dt.values()) 
# # del dt

In [58]:
# extra_features

In [59]:
# extra_features.to_csv("../output/MATCHES_recid_title_labforce_occstrng_age.csv")

### Filter age and dedup

In [60]:
extra_features.year = extra_features.year.astype('int16')

In [61]:
extra_features.year.value_counts()

year
1911    36353455
1901    32493318
1891    29509255
1881    26124585
1861    19828561
1851    17711058
Name: count, dtype: int64

In [62]:
result.reset_index()

,patent_id,patent_count,recid,year,name_jarowinkler,addr_to_use_levenshtein,mean,simple_weighted_score
0,GB188002025,0,1705473,1881,0.564408,0.140845,0.352626,0.437339
1,GB188002025,0,2303080,1911,0.564408,0.225352,0.394880,0.462691
2,GB188002025,0,4427159,1861,0.564408,0.169014,0.366711,0.445790
3,GB187600824,0,11166494,1891,0.856410,0.118812,0.487611,0.635131
4,GB188907812,0,8730659,1881,0.869519,0.115385,0.492452,0.643278
...,...,...,...,...,...,...,...,...
425,GB190725421,1,14986608,1851,0.858462,0.058824,0.458643,0.618570
426,GB190604178,1,14471429,1881,0.689947,0.000000,0.344974,0.482963
427,GB190002764,1,16088027,1851,0.861183,0.066667,0.463925,0.622828
428,GB190304313,2,10550285,1891,0.516667,0.250000,0.383333,0.436667


In [63]:
result = result.reset_index()
result = result[['patent_id', 'patent_count', 'recid', 'year', 'simple_weighted_score']].merge(extra_features, on = ['recid', 'year'])
result

,patent_id,patent_count,recid,year,simple_weighted_score,labforce,occstrng,age
0,GB188002025,0,1705473,1881,0.437339,2,JEWELLER,26.0
1,GB188002025,0,2303080,1911,0.462691,2,LABOURER,61.0
2,GB188002025,0,4427159,1861,0.445790,2,SOLDIER,999.0
3,GB187600824,0,11166494,1891,0.635131,2,AGRICULTURAL LABOURER,36.0
4,GB188907812,0,8730659,1881,0.643278,2,GARDNER,21.0
...,...,...,...,...,...,...,...,...
425,GB190522311,1,2299774,1881,0.382305,2,WOOLLEN DRAPERS ASSIST,38.0
426,GB190918539,1,26925900,1891,0.422001,2,RAILWAY CROSSING KEEPER,83.0
427,GB190604178,1,14471429,1881,0.482963,2,FARMER OF 96 ACRES EMP 2 MEN 1,27.0
428,GB190304313,2,10550285,1891,0.436667,1,,43.0


In [64]:
result.labforce.value_counts()

labforce
2    343
1     86
8      1
Name: count, dtype: int64

In [65]:
# result['simple_weighted_score'] = 0.7*result.name_jarowinkler + 0.3*result.addr_to_use_levenshtein

In [66]:
age_mask = result.age > 16
age_filtered = result[age_mask].sort_values(by = ['patent_id', 'simple_weighted_score'], ascending = False)
age_filtered

,patent_id,patent_count,recid,year,simple_weighted_score,labforce,occstrng,age
144,GB191114303,0,22156521,1881,0.387892,2,AGRICULTURAL LABOURER,31.0
378,GB191114303,0,35812507,1911,0.387636,2,SORTING CLERK AND TELEGRAPHIST,26.0
201,GB191109077,0,16400567,1891,0.652875,2,AGRICULTURAL LABOURER,58.0
397,GB191105791,0,19012617,1891,0.695753,2,TAILORESS,18.0
202,GB191103479,0,8563586,1881,0.449444,2,AGRI LAB,46.0
...,...,...,...,...,...,...,...,...
5,GB187102200,0,4266587,1861,0.624381,2,AGRICULTURAL LABOURER,62.0
301,GB187102200,0,25375222,1891,0.613449,2,FARMER,58.0
408,GB187102200,1,15205219,1911,0.507652,2,GENERAL DOMESTIC,33.0
409,GB187102200,1,13851342,1901,0.495064,2,SERVANT DOMESTIC,24.0


In [67]:
age_filtered.to_csv('../output/pat_census_age_filtered.csv', index = False)

In [68]:
age_filtered

,patent_id,patent_count,recid,year,simple_weighted_score,labforce,occstrng,age
144,GB191114303,0,22156521,1881,0.387892,2,AGRICULTURAL LABOURER,31.0
378,GB191114303,0,35812507,1911,0.387636,2,SORTING CLERK AND TELEGRAPHIST,26.0
201,GB191109077,0,16400567,1891,0.652875,2,AGRICULTURAL LABOURER,58.0
397,GB191105791,0,19012617,1891,0.695753,2,TAILORESS,18.0
202,GB191103479,0,8563586,1881,0.449444,2,AGRI LAB,46.0
...,...,...,...,...,...,...,...,...
5,GB187102200,0,4266587,1861,0.624381,2,AGRICULTURAL LABOURER,62.0
301,GB187102200,0,25375222,1891,0.613449,2,FARMER,58.0
408,GB187102200,1,15205219,1911,0.507652,2,GENERAL DOMESTIC,33.0
409,GB187102200,1,13851342,1901,0.495064,2,SERVANT DOMESTIC,24.0


In [69]:
!ls -lha ../Output/pat_census_age_filtered*.csv

ls: cannot access ../Output/pat_census_age_filtered*.csv: No such file or directory


### We don't dedup here for quality

In [ ]:
# per patent author, per year, only keep the recid with the highest mean
deduped = age_filtered.drop_duplicates(subset = ["patent_id", "patent_count", "year"], keep = 'first')
deduped.year.value_counts()

In [ ]:
# deduped[["patent_id", "patent_count"]].nunique()
deduped.drop_duplicates(subset = ["patent_id", "patent_count"]).shape[0]

In [ ]:
deduped.labforce.value_counts()

In [ ]:
# Try tracking
deduped[deduped['simple_weighted_score'] > 0.8]

In [ ]:
deduped[['patent_id', 'patent_count', 'recid', 'year', 'simple_weighted_score', 'occstrng','age']]

In [ ]:
# mask_not_growing_younger 
# mask_not_growing_younger